# Tahoe Quantum Challenge Solution
### Hadamard Hooligans

In [31]:
import random 
from collections import deque
from IPython.display import display, HTML
import json
with open('map_data.json', 'r') as file:
    map_data = json.load(file)

def get_risk():
    return 0.5

def get_neighbors(x, y, n):
    directions = [(-1,0), (1,0), (0,-1), (0,1)]
    return [(x+dx, y+dy) for dx, dy in directions if 0 <= x+dx < n and 0 <= y+dy < n]

def valid_types_around(grid, x, y, n):
    neighbors = get_neighbors(x, y, n)
    types = set(grid[nx][ny]['type'] for nx, ny in neighbors if grid[nx][ny] is not None)
    possible = []

    if types and types.issubset({'E', 'H'}):
        possible.append('H')
    if types & {'H'}:
        possible.append('E')
    if types & {'E', 'D'} and 'H' not in types:
        possible.append('D')

    return possible

def generate_grid(n, d_limit=3):
    grid = [[None for _ in range(n)] for _ in range(n)]
    start_x, start_y = random.randint(0, n-1), random.randint(0, n-1)
    grid[start_x][start_y] = {
        'type': 'E',
        'population': 0,
        'risk': 0.5,
    }
    frontier = deque([(start_x, start_y)])

    while frontier:
        x, y = frontier.popleft()
        for nx, ny in get_neighbors(x, y, n):
            if grid[nx][ny] is None:
                options = valid_types_around(grid, nx, ny, n)
                if options:
                    typ = random.choice(options)
                    pop = random.randint(500, 4000) if typ == 'H' else 0
                    risk = get_risk()

                    # grid[nx][ny] = choice
                    grid[nx][ny] = {
                        'type': typ,
                        'population': pop,
                        'risk': risk
                    }
                    frontier.append((nx, ny))

    return grid

def print_grid(grid):
    html = "<table style='border-collapse: collapse;'>"
    
    for row in grid:
        html += "<tr style='text-align: center; line-height: 20px; font-size: 15pt;'>"
        for cell in row:
            if cell['type'] in ['D', 'H', 'E']:
                if cell['type'] == 'D':
                    color = 'darkgreen'
                elif cell['type'] == 'H':
                    color = 'gray'
                elif cell['type'] == 'E':
                    color = 'green'
                html += "<td style='background-color: {}; color: white; padding: 10px; border: 1px solid black; width: 60px;'><b>{}</b><br>{}<br>{}</td>".format(color, cell['type'], cell['population'], cell['risk'])  # Dark Green for D
            else:
                html += "<td style='background-color: white; color: black; padding: 10px; border: 1px solid black; width: 60px;'>.</td>"  # Default for empty cells
            
        html += "</tr>"
    
    html += "</table>"
    
    display(HTML(html))

# Example usage
n = 8
grid = generate_grid(n)
f = json.dumps(grid)
print_grid(grid)

with open('grid.json', 'w') as file:
    file.write(f)

D00.5,D00.5,D00.5,D00.5,D00.5,E00.5,D00.5,D00.5
E00.5,D00.5,D00.5,D00.5,E00.5,H35890.5,E00.5,D00.5
H20980.5,E00.5,D00.5,E00.5,H30350.5,H15270.5,E00.5,D00.5
E00.5,H28420.5,E00.5,H12040.5,H15980.5,E00.5,D00.5,D00.5
D00.5,E00.5,D00.5,E00.5,H15100.5,E00.5,D00.5,D00.5
D00.5,D00.5,D00.5,D00.5,E00.5,D00.5,D00.5,D00.5
D00.5,D00.5,D00.5,D00.5,D00.5,D00.5,D00.5,D00.5
D00.5,D00.5,D00.5,D00.5,D00.5,D00.5,D00.5,D00.5
